In [153]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler



engine = create_engine("sqlite:///horseracing.db")
sqlite_connection = engine.connect()


query = """

select *, case when plc_clean in ('1','2','3') then 1 else 0 end as Target_Value
FROM
(
select *, case when length(plc) > 3 then rtrim(substr(plc, 1, 1)) else plc END as plc_clean
from tbl_racing_results
where plc not in ('WV', 'WX', 'UR', 'PU', 'WV-A', 'FE', 'WX-A', 'DNF', 'TNP')
)


"""


df_sql = pd.read_sql(query, engine)
df_sql

,row,plc,horseno,horse,jockey,trainer,actualwt,declarwt,draw,lbw,...,horse_ch,jockey_ch,trainer_ch,class_ch,going_ch,handicap_ch,course_ch,venue_ch,plc_clean,Target_Value
0,0,1,10.0,NATURAL EIGHT(S243),C Y Ho,C Fownes,119.0,1037.0,1.0,-,...,威百利(S243),何澤堯,方嘉柏,1,好地至快地,畢拿山讓賽,草地 - A 賽道,跑馬地,1,1
1,0,1,7.0,SOLAR HEI HEI(T242),B Prebble,D J Hall,125.0,1077.0,11.0,-,...,太陽喜喜(T242),柏寶,賀賢,1,好地至快地,香港大學基金讓賽,草地 - A+3 賽道,沙田,1,1
2,0,1,3.0,GRAN BOLIVAR(S165),H N Wong,Y S Tsui,121.0,950.0,9.0,-,...,王將(S165),黃皓楠,徐雨石,1,好地至快地,麗安讓賽,草地 - B 賽道,跑馬地,1,1
3,0,1,11.0,PEOPLE'S KNIGHT(T305),G Mosse,J Moore,120.0,1149.0,4.0,-,...,人民武士(T305),巫斯義,約翰摩亞,1,好地至快地,ＢＲＥＧＵＥＴ平磅賽,草地 - B+2 賽道,沙田,1,1
4,0,1,2.0,SIR JOHN(P344),B Prebble,W Y So,131.0,1158.0,10.0,-,...,至尊駿爵(P344),柏寶,蘇偉賢,1,好地至快地,宏德讓賽,草地 - C 賽道,跑馬地,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13878,13,14,11.0,DISCIPLES TWELVE(P018),K Teetan,C Fownes,119.0,1107.0,1.0,10-1/4,...,十二門徒(P018),田泰安,方嘉柏,1,好地,北京讓賽,草地 - A+3 賽道,沙田,14,0
13879,13,14,10.0,SPEEDY LONGWAH(S242),T H So,C S Shum,124.0,1124.0,13.0,50,...,金滙龍華(S242),蘇狄雄,沈集成,1,好地,其士安老院舍讓賽,草地 - C 賽道,沙田,14,0
13880,13,14,12.0,MARVEL TRIBE(S142),K C Ng,K W Lui,110.0,1118.0,2.0,14-1/4,...,萬馬奔騰(S142),吳嘉晉,呂健威,1,好地至快地,大帽山讓賽,草地 - B+2 賽道,沙田,14,0
13881,13,14,13.0,LET US WIN(V381),K K Chiong,J Moore,108.0,1111.0,3.0,17,...,共創更好(V381),蔣嘉琦,約翰摩亞,1,好地至快地,氹仔讓賽,草地 - C 賽道,沙田,14,0


## Remove Chinese Language Columns

In [154]:
df_sql.drop('horse_ch', axis=1, inplace=True)
df_sql.drop('jockey_ch', axis=1, inplace=True)
df_sql.drop('trainer_ch', axis=1, inplace=True)
df_sql.drop('class_ch', axis=1, inplace=True)
df_sql.drop('going_ch', axis=1, inplace=True)
df_sql.drop('handicap_ch', axis=1, inplace=True)
df_sql.drop('course_ch', axis=1, inplace=True)
df_sql.drop('venue_ch', axis=1, inplace=True)
df_sql.drop('lbw', axis=1, inplace=True)

In [155]:
df_sql

,row,plc,horseno,horse,jockey,trainer,actualwt,declarwt,draw,runningpos,...,raceno,class,distance,going,handicap,course,stake,venue,plc_clean,Target_Value
0,0,1,10.0,NATURAL EIGHT(S243),C Y Ho,C Fownes,119.0,1037.0,1.0,4 3 1,...,1,5,1200,GOOD TO FIRM,MOUNT BUTLER HANDICAP,TURF - A COURSE,575000,Happy Valley,1,1
1,0,1,7.0,SOLAR HEI HEI(T242),B Prebble,D J Hall,125.0,1077.0,11.0,9 8 1,...,1,4,1200,GOOD TO FIRM,HKU FOUNDATION HANDICAP,TURF - A+3 COURSE,760000,Sha Tin,1,1
2,0,1,3.0,GRAN BOLIVAR(S165),H N Wong,Y S Tsui,121.0,950.0,9.0,1 1 1,...,1,4,1200,GOOD TO FIRM,LAI ON HANDICAP,TURF - B COURSE,760000,Happy Valley,1,1
3,0,1,11.0,PEOPLE'S KNIGHT(T305),G Mosse,J Moore,120.0,1149.0,4.0,4 4 1,...,1,1,1200,GOOD TO FIRM,BREGUET PLATE,TURF - B+2 COURSE,800000,Sha Tin,1,1
4,0,1,2.0,SIR JOHN(P344),B Prebble,W Y So,131.0,1158.0,10.0,1 1 1,...,1,5,1000,GOOD TO FIRM,WANG TAK HANDICAP,TURF - C COURSE,575000,Happy Valley,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13878,13,14,11.0,DISCIPLES TWELVE(P018),K Teetan,C Fownes,119.0,1107.0,1.0,10 11 14,...,11,2,1200,GOOD,BEIJING HANDICAP,TURF - A+3 COURSE,1750000,Sha Tin,14,0
13879,13,14,10.0,SPEEDY LONGWAH(S242),T H So,C S Shum,124.0,1124.0,13.0,14 14 14 14,...,11,2,1400,GOOD,CHEVALIER SENIOR HOUSING HANDICAP,TURF - C COURSE,1750000,Sha Tin,14,0
13880,13,14,12.0,MARVEL TRIBE(S142),K C Ng,K W Lui,110.0,1118.0,2.0,3 7 14,...,11,2,1200,GOOD TO FIRM,TAI MO SHAN HANDICAP,TURF - B+2 COURSE,1750000,Sha Tin,14,0
13881,13,14,13.0,LET US WIN(V381),K K Chiong,J Moore,108.0,1111.0,3.0,4 1 1 4 14,...,11,2,1800,GOOD TO FIRM,TAIPA HANDICAP,TURF - C COURSE,1750000,Sha Tin,14,0


## Create a Weight Class

In [156]:
def weight_class(row):
    if row['actualwt'] < 116:
        return 'Lightweight'
    elif row['actualwt'] >= 116 and row['actualwt'] <= 125:
        return 'Middleweight'
    elif row['actualwt'] > 125:
        return 'HeavyWeight'

df_sql['weight_class'] = df_sql.apply (lambda row: weight_class(row), axis=1)

## Create Horse's Last Race Statistics

In [157]:
query2 = """select tbl_racing_results.plc, tbl_racing_results.horse, tbl_racing_results.date, tempp.Placing, tempp.Winning
from tbl_racing_results
left join tbl_horse_info 
	on  tbl_racing_results.horse = tbl_horse_info.horse
inner join
(
    select Placing, Winning, count2, horse, date as date_2
    from
        (
        select *, case when tbl_racing_results.plc in (1,2,3) then 1 else 0 end as Placing, case when tbl_racing_results.plc in (1) then 1 else 0 end as Winning, ROW_NUMBER() OVER (
        PARTITION BY tbl_racing_results.horse
        ORDER BY tbl_racing_results.date asc)  as count2
        from tbl_racing_results
        )
) as tempp
on tbl_racing_results.date > tempp.date_2 and tbl_racing_results.horse = tempp.horse
order by tbl_racing_results.horse, date asc
"""


df = pd.read_sql(query2, engine)
h = df.groupby(['horse', 'date'])
h
h = h.tail(3)
win_rate = h.groupby(['plc','horse', 'date']).sum().reset_index()

new_data_df = df_sql.merge(win_rate, how='left', left_on = ['horse', 'date'], right_on = ['horse', 'date'])
new_data_df['Placing'] = new_data_df['Placing'].fillna(0)
new_data_df['Winning'] = new_data_df['Winning'].fillna(0)

## Create Jockey's Last Race Stats

In [158]:
query2 = """select tbl_racing_results.plc, tbl_racing_results.jockey, tbl_racing_results.date, tempp.Jockey_Placing, tempp.Jockey_Winning
from tbl_racing_results
left join tbl_horse_info 
	on  tbl_racing_results.horse = tbl_horse_info.horse
inner join
(
    select Jockey_Placing, Jockey_Winning, count2, jockey, date as date_2
    from
        (
        select *, case when tbl_racing_results.plc in (1,2,3) then 1 else 0 end as Jockey_Placing, case when tbl_racing_results.plc in (1) then 1 else 0 end as Jockey_Winning, ROW_NUMBER() OVER (
        PARTITION BY tbl_racing_results.jockey
        ORDER BY tbl_racing_results.date asc)  as count2
        from tbl_racing_results
        )
) as tempp
on tbl_racing_results.date > tempp.date_2 and tbl_racing_results.jockey = tempp.jockey
order by tbl_racing_results.jockey, date asc
"""


df = pd.read_sql(query2, engine)
j = df.groupby(['jockey', 'date'])
j = j.tail(3)
jockey_win_rate = j.groupby(['plc','jockey', 'date']).sum().reset_index()
jockey_win_rate
new_data_df = new_data_df.merge(jockey_win_rate, how='left', left_on = ['jockey', 'date'], right_on = ['jockey', 'date'])
new_data_df['Jockey_Placing'] = new_data_df['Jockey_Placing'].fillna(0)
new_data_df['Jockey_Winning'] = new_data_df['Jockey_Winning'].fillna(0)

## Create Position Score

In [159]:
query_pos = """
select tbl_racing_results.horse, tbl_racing_results.date, date_2, coalesce(tempp.runningpos_clean,0) as runningpos_clean
from tbl_racing_results
left join
(
select horse, date as date_2, ltrim(substr(runningpos,-6, 6)) as runningpos_clean
from tbl_racing_results
 
) as tempp
on tbl_racing_results.date > tempp.date_2 and tbl_racing_results.horse = tempp.horse
order by tbl_racing_results.horse, date asc

"""

running_pos = pd.read_sql(query_pos, engine)

import re
def pos_cal(row):
    counter = 0
    row_list = re.split("\s", str(row['runningpos_clean']))
    for x in row_list:
        x_int = int(x)
        if x_int>=1 and x_int<5:
            counter+= 4
        elif x_int>=5 and x_int<9:
            counter+= 2
        else:
            counter+=0
    return counter


running_pos['pos_counter'] = running_pos.apply (lambda row: pos_cal(row), axis=1)
pos = running_pos.groupby(['horse', 'date'])
pos = pos.tail(2)
pos_status = pos.groupby(['horse','date']).sum().reset_index()

new_data_df = new_data_df.merge(pos_status, how='left', left_on = ['horse', 'date'], right_on = ['horse', 'date'])



## Create the Jmoreria identifier. This Jockey had over a 50% win rate

In [103]:
def J_moreria_effect(row):
    if row['jockey'] == 'J Moreira':
        return 1
    else:
        return 0

new_data_df['J_Moreira_y_n'] = new_data_df.apply(lambda row: J_moreria_effect(row), axis=1)

## Example of Horse's Last Game Stats

In [185]:
test = new_data_df[(new_data_df['date']=='2016-03-31') & (new_data_df['raceno'] == 8)][['horse', 'Placing', 'Winning']]


alt.Chart(test).mark_bar().encode(
    x=alt.X('Placing', axis=alt.Axis(title="Horse's Show Show from Last 3 Games")),
    y='horse:O',
#    color='year:N',
#    column='site:N'
)




alt.Chart(...)

## Drop Additional Rows  and Sort Values

In [104]:
new_data_df.drop('stake', axis=1, inplace=True)
new_data_df.drop('trainer', axis=1, inplace=True)
#new_data_df.drop('jockey', axis=1, inplace=True)
new_data_df.drop('runningpos', axis=1, inplace=True)
new_data_df.drop('handicap', axis=1, inplace=True)
new_data_df.drop('going', axis=1, inplace=True)
#new_data_df.drop('venue', axis=1, inplace=True)
new_data_df.drop('row', axis=1, inplace=True)
new_data_df.drop('plc_x', axis=1, inplace=True)
new_data_df.drop('plc_y', axis=1, inplace=True)
#new_data_df.drop('horseno', axis=1, inplace=True)
new_data_df.drop('course', axis=1, inplace=True)
new_data_df.drop('finishtime', axis=1, inplace=True)
#new_data_df.drop('horse', axis=1, inplace=True)
new_data_df.drop('plc_clean', axis=1, inplace=True)


new_data_df = new_data_df.sort_values(by=['date', 'raceno'])
#new_data_df.drop('date', axis=1, inplace=True)
#new_data_df.drop('raceno', axis=1, inplace=True)


In [32]:
new_data_df = new_data_df.sort_values(by=['raceno', 'date'])
new_data_df[-14:]
new_data_df.drop('date', axis=1, inplace=True)
new_data_df.drop('raceno', axis=1, inplace=True)

## Create dummy variable

In [33]:
new_data_df = pd.get_dummies(new_data_df, columns = ['weight_class'])



## Pickle Data

In [12]:
import pickle
with open('mypickle.pickle', 'wb') as f:
    pickle.dump(new_data_df, f)

In [32]:
import seaborn as sns
#sns.pairplot(new_data_df, hue='Target_Value')

In [59]:
from sklearn.metrics import accuracy_score
import xgboost as xgb

def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())

In [144]:
import altair as alt

horse_dict = {"FLYING MOOCHI": .22 ,
"MARVEL TRIBE": .05,
"PERPETUAL TREASURE":  .09,
"CLEVER BEAVER": .43, 
"LINE SEEKER": .40, 
"SO FAST": .10,
"MOST BEAUTIFUL": .14,
"MERION": .56,
"MIDNIGHT RATTLER": .47,
"SECRET COMMAND": .11,
"MR STUNNING": .81,
"INVINCIBLE DRAGON": .57,
"TURF SPRINT": .37,
"KEY WITNESS": .37
                }
df_race = pd.DataFrame.from_dict(horse_dict, orient='index',  
                                 columns=['Prediction']).reset_index()

df_race
bars = alt.Chart(df_race).mark_bar().encode(
   x='Prediction',
    y= 'index',
    color=alt.condition(
        alt.datum.Prediction > .50, # If the year is 1810 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('lightgray')   # And if it's not true it sets the bar steelblue.
    ))
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Prediction'
)
#).properties(height=300)
              
(bars + text).properties(height=300, width = 600)


alt.LayerChart(...)

In [149]:
distribution_dic = {'Showing': 24, 'Not Showing': 76}

df_dist = pd.DataFrame.from_dict(distribution_dic, orient='index',  
                                 columns=['Percentage']).reset_index()

bars = alt.Chart(df_dist).mark_bar().encode(
   x='index',
    y= 'Percentage',
    color=alt.condition(
        alt.datum.Percentage > 50, # If the year is 1810 this test returns True,
        alt.value('red'),     # which sets the bar orange.
        alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
    ))

#).properties(height=300)
              
(bars).properties(height=250, width = 400)

alt.Chart(...)

In [152]:
import altair as alt


horse_dict = {"IMPERIAL CHAMPION": .09 ,
"AMBER DRAGON": .22,
"BORNTOACHIEVE":  .21,
"WONDERFUL JOURNEY": .03, 
"HELLA HEDGE": .84, 
"GENERAL IRON": .25,
"GOLDEN DEER": .45,
"FOREVER RED": .25,
"PEACE N PROSPERITY": .64,
"LINE SEEKER": .42,
"RAINBOW FIGHTER": .51,
"GRACYDAD": .57
                }
df_race = pd.DataFrame.from_dict(horse_dict, orient='index',  
                                 columns=['Prediction']).reset_index()

df_race
bars = alt.Chart(df_race).mark_bar().encode(
   x='Prediction',
    y= 'index',
    color=alt.condition(
        alt.datum.Prediction > .51, # If the year is 1810 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('lightgray')   # And if it's not true it sets the bar steelblue.
    ))
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Prediction'
)
#).properties(height=300)
              
(bars + text).properties(height=300, width = 600)


alt.LayerChart(...)